# CAD Assignment - Rectangle Analysis
## Complete Solution for Q1-Q6

This notebook contains all the code for analyzing rectangles including overlap detection, containment analysis, and abutting detection.

## 1. Core Classes and Utilities

In [ ]:
import time
import math as m
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from typing import List, Tuple, Set

# LineEquation class: represents ax + by = c
class LineEquation:
    """Represents a line equation of the form ax + by = c"""
    def __init__(self, a: int, b: int, c: int):
        self.a = a
        self.b = b
        self.c = c
    
    def slope(self):
        if self.b == 0:
            return float('inf')
        return -self.a / self.b


# Block class: represents a rectangle with geometric properties
class block:
    """Class representing a rectangular block with geometric properties"""
    def __init__(self, block_id: int, colour_code: str, height: float, width: float, 
                 left_bottom_coord: Tuple[float, float], west: LineEquation, south: LineEquation):
        self.block_id = block_id
        self.colour_code = colour_code
        self.height = height
        self.width = width
        self.left_bottom_coord = left_bottom_coord
        self.west = west
        self.south = south

    def area(self):
        return self.height * self.width
    
    def east(self):
        return LineEquation(self.west.a, self.west.b, self.west.c + self.width)
    
    def north(self):
        return LineEquation(self.south.a, self.south.b, self.south.c + self.height)
    
    def left_top_coord(self):
        return (self.left_bottom_coord[0], self.left_bottom_coord[1] + self.height)
    
    def right_bottom_coord(self):
        return (self.left_bottom_coord[0] + self.width, self.left_bottom_coord[1])
    
    def right_top_coord(self):
        return (self.left_bottom_coord[0] + self.width, self.left_bottom_coord[1] + self.height)

## 2. Data Extraction and Plotting Functions

In [ ]:
# Color list for plotting
mcolors_list = list(mcolors.CSS4_COLORS.keys())

def equation_creator(x1, y1, x2, y2):
    """Create line equation coefficients for a line passing through (x1,y1) and (x2,y2)"""
    a = y2 - y1
    b = x1 - x2
    c = a * x1 + b * y1
    return (a, b, c)

def ext_blocks(filename):
    """Extract blocks from input file"""
    start_time = time.time()
    blocks_list = []
    colour_code = 0
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue
                
                values = line.replace('{', '').replace('}', '').split(',')
                values = values[0:5]
                values = [int(v.strip()) for v in values]
                
                if len(values) == 5:
                    rect_id, x1, y1, x2, y2 = values
                    height = m.fabs(y2 - y1)
                    width = m.fabs(x2 - x1)
                    colour_code += 20
                    rect = block(block_id=rect_id, colour_code=mcolors_list[colour_code % len(mcolors_list)], 
                                 height=height, width=width, left_bottom_coord=(x1, y1), 
                                 west=LineEquation(*equation_creator(x1, y1, x1, y2)), 
                                 south=LineEquation(*equation_creator(x1, y1, x2, y1)))
                    blocks_list.append(rect)
                else:
                    print(f"Warning: Line '{line}' does not contain exactly 5 values. Skipping.")
    
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return [], 0
    
    elapsed_time = time.time() - start_time
    print(f"\nData Extraction Summary:")
    print(f"Rectangles loaded: {len(blocks_list)}")
    print(f"Time taken for extraction: {elapsed_time:.4f} seconds")
    
    return blocks_list, elapsed_time

def plot_blocks(blocks_list, point=None, show=True, title="Block Placement"):
    """Plot blocks on a graph"""
    start_time = time.time()
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for block_obj in blocks_list:
        x, y = tuple(block_obj.left_bottom_coord)
        rect = patches.Rectangle((x, y), block_obj.width, block_obj.height, 
                                 linewidth=4, edgecolor=block_obj.colour_code, facecolor='none')
        ax.add_patch(rect)
        # Add block ID label
        ax.text(x + block_obj.width/2, y + block_obj.height/2, str(block_obj.block_id), 
               ha='center', va='center', fontsize=10, fontweight='bold')
    
    if point:
        ax.plot(point[0], point[1], 'ro', markersize=10)
    
    ax.relim()
    ax.autoscale_view()
    
    elapsed_time = time.time() - start_time
    print(f"\nTime taken for block placement: {elapsed_time:.4f} seconds")
    
    ax.set_aspect('auto', adjustable='box')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title(title)
    plt.grid()
    
    if show:
        plt.show()
    
    return elapsed_time

## 3. Printing Utilities

In [ ]:
def printer(blocks_list):
    """Print block IDs in format {id1, id2, id3, ...}"""
    if blocks_list:
        ids = ', '.join(str(item.block_id if hasattr(item, 'block_id') else item) for item in blocks_list)
        print(f"{{{ids}}}")
    else:
        print("{}")

def print_set(s):
    """Print nested sets in format {{...}, {...}, {...}}"""
    if s:
        formatted_sets = ['{' + ', '.join(str(item) for item in x) + '}' for x in s]
        print('{' + ', '.join(formatted_sets) + '}')
    else:
        print("{}")

## 4. Overlap Detection Functions

In [ ]:
def overlap_check(block1, block2):
    """Check if two blocks overlap (including touching edges)
    Returns True if blocks overlap, False otherwise"""
    x1_min = block1.left_bottom_coord[0]
    y1_min = block1.left_bottom_coord[1]
    x1_max = x1_min + block1.width
    y1_max = y1_min + block1.height
    
    x2_min = block2.left_bottom_coord[0]
    y2_min = block2.left_bottom_coord[1]
    x2_max = x2_min + block2.width
    y2_max = y2_min + block2.height
    
    return not (x1_max <= x2_min or x1_min >= x2_max or y1_max <= y2_min or y1_min >= y2_max)

def partial_overlap_check(block1, block2):
    """Check type of overlap between two blocks.
    Returns: (overlap_type, direction)
        0: No overlap
        1: Partial overlap
        2: Full overlap (block1 fully inside block2)
        3: Abutting (touching at one edge only)
    """
    x1_min = block1.left_bottom_coord[0]
    y1_min = block1.left_bottom_coord[1]
    x1_max = x1_min + block1.width
    y1_max = y1_min + block1.height
    
    x2_min = block2.left_bottom_coord[0]
    y2_min = block2.left_bottom_coord[1]
    x2_max = x2_min + block2.width
    y2_max = y2_min + block2.height
    
    # Check if blocks intersect
    blocks_intersect = not (x1_max < x2_min or x1_min > x2_max or y1_max < y2_min or y1_min > y2_max)
    
    # Check if block1 is fully inside block2
    block1_inside_block2 = (x2_min <= x1_min and x1_max <= x2_max and y2_min <= y1_min and y1_max <= y2_max)
    
    # Check abutting direction
    east = x1_max == x2_min
    west = x1_min == x2_max
    north = y1_max == y2_min
    south = y1_min == y2_max
    abutting = (
        (east or west) and (y1_min < y2_max and y1_max > y2_min)
    ) or (
        (north or south) and (x1_min < x2_max and x1_max > x2_min)
    )
    direction = east*1 + west*2 + north*3 + south*4
    
    if not blocks_intersect and not abutting:
        return 0, 0
    elif abutting and not block1_inside_block2:
        return 3, direction
    elif block1_inside_block2:
        return 2, 0
    else:
        return 1, 0

def is_block_inside(container, contained):
    """Check if 'contained' block is fully inside 'container' block"""
    x_cont_min = container.left_bottom_coord[0]
    y_cont_min = container.left_bottom_coord[1]
    x_cont_max = x_cont_min + container.width
    y_cont_max = y_cont_min + container.height
    
    x_con_min = contained.left_bottom_coord[0]
    y_con_min = contained.left_bottom_coord[1]
    x_con_max = x_con_min + contained.width
    y_con_max = y_con_min + contained.height
    
    return (x_cont_min <= x_con_min and x_con_max <= x_cont_max and 
            y_cont_min <= y_con_min and y_con_max <= y_cont_max)

## Q1: Display All Rectangles

In [ ]:
def q1_display_all():
    """Q1: Display all rectangles graphically"""
    filename = input("Enter the filename: ")
    blocks_list, t1 = ext_blocks(filename)
    t2 = plot_blocks(blocks_list)
    print(f"\nTotal time taken for Q1: {t1+t2:.4f} seconds")

# Uncomment to run:
# q1_display_all()

## Q2: Point in Rectangle

In [ ]:
def point_check(block_obj, x, y):
    """Check if point (x,y) is inside block"""
    return ((block_obj.left_bottom_coord[0] <= x <= block_obj.left_bottom_coord[0] + block_obj.width) and 
            (block_obj.left_bottom_coord[1] <= y <= block_obj.left_bottom_coord[1] + block_obj.height))

def q2_point_in_blocks():
    """Q2: Find which rectangles contain a given point"""
    filename = input("Enter the filename: ")
    blocks_list, t1 = ext_blocks(filename)
    x1 = int(input("Enter x coordinate: "))
    y1 = int(input("Enter y coordinate: "))
    
    start_time = time.time()
    point_in_block_list = []
    for block_obj in blocks_list:
        if point_check(block_obj, x1, y1):
            point_in_block_list.append(block_obj.block_id)
    
    printer(point_in_block_list)
    elapsed_time = time.time() - start_time
    print(f"\nTime taken for data processing in Q2: {elapsed_time:.4f} seconds")
    
    t2 = plot_blocks(blocks_list, (x1, y1))
    print(f"\nTotal time taken for Q2: {t1+elapsed_time+t2:.4f} seconds")

# Uncomment to run:
# q2_point_in_blocks()

## Q3: Non-Overlapping Rectangles

In [ ]:
def q3_no_overlap():
    """Q3: Find rectangles that don't overlap with any other rectangle"""
    filename = input("Enter the filename: ")
    blocks_list, t1 = ext_blocks(filename)
    
    start_time = time.time()
    overlapping_blocks = set()
    n = len(blocks_list)
    
    for i in range(n):
        for j in range(i + 1, n):
            if overlap_check(blocks_list[i], blocks_list[j]):
                overlapping_blocks.add(blocks_list[i].block_id)
                overlapping_blocks.add(blocks_list[j].block_id)
    
    non_overlap_list = [b for b in blocks_list if b.block_id not in overlapping_blocks]
    printer(non_overlap_list)
    elapsed_time = time.time() - start_time
    print(f"\nTime taken for data processing in Q3: {elapsed_time:.4f} seconds")
    
    t2 = plot_blocks(blocks_list, show=False)
    t3 = plot_blocks(non_overlap_list, show=False, title="Non-Overlapping Blocks")
    plt.show()
    print(f"\nTotal time taken for Q3: {t1+elapsed_time+t2+t3:.4f} seconds")

# Uncomment to run:
# q3_no_overlap()

## Q4: Partially Overlapping Rectangles

In [ ]:
def q4_partial_overlap():
    """Q4: Find rectangles that partially overlap with others"""
    filename = input("Enter the filename: ")
    blocks_list, t1 = ext_blocks(filename)
    
    start_time = time.time()
    overlapping_blocks = list()
    all_overlapping_blocks = set()
    n = len(blocks_list)
    
    for i in range(n):
        overlap_list = [blocks_list[i].block_id]
        for j in range(n):
            if (i != j) and (blocks_list[j].block_id not in all_overlapping_blocks):
                overlap_type, ignore = partial_overlap_check(blocks_list[i], blocks_list[j])
                if overlap_type == 1:
                    overlap_list.append(blocks_list[j].block_id)
                    all_overlapping_blocks.add(blocks_list[i])
                    all_overlapping_blocks.add(blocks_list[j])
        if len(overlap_list) > 1:
            overlapping_blocks.append(overlap_list)
    
    elapsed_time = time.time() - start_time
    print_set(overlapping_blocks)
    print(f"\nTime taken for data processing in Q4: {elapsed_time:.4f} seconds")
    
    t2 = plot_blocks(blocks_list, show=False)
    t3 = plot_blocks(all_overlapping_blocks, show=False, title="Partially Overlapping Blocks")
    plt.show()
    print(f"\nTotal time taken for Q4: {t1+elapsed_time+t2+t3:.4f} seconds")

# Uncomment to run:
# q4_partial_overlap()

## Q5: Fully Contained Rectangles

In [ ]:
def q5_fully_contained():
    """Q5: Find rectangles that fully contain other rectangles"""
    filename = input("Enter the filename: ")
    blocks_list, t1 = ext_blocks(filename)
    
    start_time = time.time()
    overlapping_blocks = list()
    all_overlapping_blocks = set()
    n = len(blocks_list)
    
    for i in range(n):
        overlap_list = [blocks_list[i].block_id]
        for j in range(n):
            if (i != j) and (blocks_list[j].block_id not in all_overlapping_blocks):
                if is_block_inside(blocks_list[i], blocks_list[j]):
                    overlap_list.append(blocks_list[j].block_id)
                    all_overlapping_blocks.add(blocks_list[i])
                    all_overlapping_blocks.add(blocks_list[j])
        if len(overlap_list) > 1:
            overlapping_blocks.append(overlap_list)
    
    elapsed_time = time.time() - start_time
    print_set(overlapping_blocks)
    print(f"\nTime taken for data processing in Q5: {elapsed_time:.4f} seconds")
    
    t2 = plot_blocks(blocks_list, show=False)
    t3 = plot_blocks(all_overlapping_blocks, show=False, title="Fully Contained Blocks")
    plt.show()
    print(f"\nTotal time taken for Q5: {t1+elapsed_time+t2+t3:.4f} seconds")

# Uncomment to run:
# q5_fully_contained()

## Q6: Abutting Rectangles

In [ ]:
def q6_abutting():
    """Q6: Find rectangles that are abutting (touching at one edge)"""
    filename = input("Enter the filename: ")
    blocks_list, t1 = ext_blocks(filename)
    
    start_time = time.time()
    overlapping_blocks = list()
    all_overlapping_blocks = set()
    n = len(blocks_list)
    
    for i in range(n):
        for j in range(n):
            if (i != j) and (blocks_list[j].block_id not in all_overlapping_blocks):
                overlap_type, direction = partial_overlap_check(blocks_list[i], blocks_list[j])
                if overlap_type == 3:
                    overlap_list = [blocks_list[i].block_id]
                    if direction == 1:
                        overlap_list.append('e')
                    elif direction == 2:
                        overlap_list.append('w')
                    elif direction == 3:
                        overlap_list.append('n')
                    else:
                        overlap_list.append('s')
                    overlap_list.append(blocks_list[j].block_id)
                    all_overlapping_blocks.add(blocks_list[i])
                    all_overlapping_blocks.add(blocks_list[j])
                    overlapping_blocks.append(overlap_list)
    
    elapsed_time = time.time() - start_time
    print_set(overlapping_blocks)
    print(f"\nTime taken for data processing in Q6: {elapsed_time:.4f} seconds")
    
    t2 = plot_blocks(blocks_list, show=False)
    t3 = plot_blocks(all_overlapping_blocks, show=False, title="Abutting Blocks")
    plt.show()
    print(f"\nTotal time taken for Q6: {t1+elapsed_time+t2+t3:.4f} seconds")

# Uncomment to run:
# q6_abutting()

## Main Execution

Uncomment the function you want to run below:

In [ ]:
# Choose which question to run by uncommenting one of the lines below:

# q1_display_all()
# q2_point_in_blocks()
# q3_no_overlap()
# q4_partial_overlap()
# q5_fully_contained()
# q6_abutting()